In [2]:
from lxml import etree as ET

def merge_tspan_attributes(parent_tspan, child_tspan):
    # Merge class attributes
    parent_class = parent_tspan.get('class', '')
    child_class = child_tspan.get('class', '')
    if child_class:
        parent_tspan.set('class', f"{parent_class} {child_class}".strip())

    # Merge style attributes
    parent_style = parent_tspan.get('style', '')
    child_style = child_tspan.get('style', '')
    if child_style:
        parent_tspan.set('style', f"{parent_style};{child_style}".strip(';'))

    # Merge other attributes except id
    for attr, value in child_tspan.attrib.items():
        if attr not in ['id', 'class', 'style']:
            parent_tspan.set(attr, value)

def flatten_tspan(tspan):
    # Loop through child tspans and merge them into the parent tspan
    for child in list(tspan):
        if child.tag.endswith('tspan'):
            merge_tspan_attributes(tspan, child)
            # Append text from the child tspan to the parent tspan
            if child.text:
                tspan.text = (tspan.text or '') + child.text

            # Remove child tspan after merging
            tspan.remove(child)

def process_svg(file_path):
    # Parse the SVG file
    parser = ET.XMLParser(remove_blank_text=True)
    tree = ET.parse(file_path, parser)
    root = tree.getroot()

    # Find all <text> elements inside the group with id="Texte"
    nsmap = root.nsmap  # Preserve the SVG namespaces
    texte_group = root.find(".//{*}g[@id='Texte']", namespaces=nsmap)
    
    if texte_group is not None:
        for text_elem in texte_group.findall('{*}text', namespaces=nsmap):
            # Process all tspans in each <text> element
            for tspan_elem in text_elem.findall('{*}tspan', namespaces=nsmap):
                flatten_tspan(tspan_elem)

    # Save the modified SVG file with proper XML declaration
    with open('modified_svg.svg', 'wb') as f:
        f.write(ET.tostring(root, pretty_print=True, xml_declaration=True, encoding='UTF-8', doctype='<!DOCTYPE svg>'))

# Example usage
process_svg('docxpand/templates/pp_td3_d/front.svg')
